<a href="https://colab.research.google.com/github/zelestik/Big-open-data-labs/blob/main/%D0%9B%D0%A02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
# from sklearn.preprocessing import LabelEncoder
# from sklearn.preprocessing import OneHotEncoder


df = pd.read_csv('winequality-red.csv', delimiter=";")

# 1. Удалить пропуски
df.dropna() 
# 2. Унифицировать качественные признаки

# Качественные признаки отсутствуют

# 3. Устранить выбросы

# Списки для хранения мат ожидания и среднеквадратического отклонения по столбцам
mList = []
sList = []
for col in df:
  mList.append(np.median(df[col]))
  sList.append(np.std(df[col]))
  # Для каждой строки проверяем вхождение в доверительный интервал по текущему столбцу
  for row in df[col]:
    if (row > (mList[len(mList)-1] + 3* sList[len(sList)-1])) or (row < (mList[len(mList)-1] - 3* sList[len(sList)-1])):
      row = np.NaN()
df.dropna() 

# 4. Выровнять по объёму классы

# перемешаем дата-сет (чтобы были удалены случайные строки)
df = df.sample(frac=1) 
# посчитаем количество элементов для каждой метки
array = df['quality'].value_counts() 
# метки, которым соответствует менее 10% элементов дата-сета - удалим
labels_to_delete = array[array < (len(df) * 0.1)]
for label in labels_to_delete.index:
  df = df[df['quality'] != label]
array = array[array > (len(df) * 0.1)] 
min = array.min()

# Для каждой метки оставим количество записей соответствующее, 
# минимальному количеству записей для метки
df = df.groupby('quality').head(min)

# 5. Копия данных
df.to_csv("DataSetCopy.csv", sep=';')

# 6. Преобразовать качественные признаки к количественным

# Качественные признаки отсутствуют в датасете

# 6.1. dt - LabelEncoder
# 6.2 knn - OneHotEncoder

# 8. Нормализация
ss = StandardScaler()
# Нормализуем
scale = ss.fit_transform(df.loc[:, df.columns != 'quality'])
# Исключаем поле с меткой
cols = df.columns.to_list()
cols.remove('quality')
# Приводим к виду датасета
new_df = pd.DataFrame(scale, index=df.index, columns=cols)
# Добавляем метку к нормализованному датасетами
new_df = new_df.merge(df['quality'], left_index=True, right_index=True)

# Сохраняем полученный результат
new_df.to_csv("DataSetFormated.csv", sep=';')

